In [ ]:
!sudo apt install nfs-common
!mkdir /content/NASData
!mount -t nfs 163.15.174.171:/volume2/HuangLab /content/NASData  -o nolock

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  keyutils libnfsidmap1 libwrap0 rpcbind
Suggested packages:
  open-iscsi watchdog
The following NEW packages will be installed:
  keyutils libnfsidmap1 libwrap0 nfs-common rpcbind
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 429 kB of archives.
After this operation, 1,559 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libnfsidmap1 amd64 1:2.6.1-1ubuntu1.2 [42.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libwrap0 amd64 7.6.q-31build2 [47.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 rpcbind amd64 1.2.6-2build1 [46.6 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 keyutils amd64 1.6.1-2ubuntu3 [50.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 nfs-common amd64 1:2.6.1-1ubuntu1.2 [241 

In [ ]:
#Show CUDA version
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Mon Mar 11 07:30:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  7

In [ ]:
!wget https://cdn.oxfordnanoportal.com/software/analysis/dorado-0.3.2-linux-x64.tar.gz -O dorado.tar.gz
!tar -xvf dorado.tar.gz

--2024-03-11 07:30:22--  https://cdn.oxfordnanoportal.com/software/analysis/dorado-0.3.2-linux-x64.tar.gz
Resolving cdn.oxfordnanoportal.com (cdn.oxfordnanoportal.com)... 13.249.141.56, 13.249.141.25, 13.249.141.19, ...
Connecting to cdn.oxfordnanoportal.com (cdn.oxfordnanoportal.com)|13.249.141.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2267666589 (2.1G) [application/x-tar]
Saving to: ‘dorado.tar.gz’

dorado.tar.gz       100%[===================>]   2.11G   231MB/s    in 13s     

2024-03-11 07:30:35 (173 MB/s) - ‘dorado.tar.gz’ saved [2267666589/2267666589]

dorado-0.3.2-linux-x64/bin/
dorado-0.3.2-linux-x64/bin/dorado
dorado-0.3.2-linux-x64/lib/
dorado-0.3.2-linux-x64/lib/libcudart.so.11.0
dorado-0.3.2-linux-x64/lib/libnvToolsExt-847d78f2.so.1
dorado-0.3.2-linux-x64/lib/libcudnn_adv_infer.so.8
dorado-0.3.2-linux-x64/lib/libsz.so
dorado-0.3.2-linux-x64/lib/libcublasLt.so.11
dorado-0.3.2-linux-x64/lib/libnvrtc.so
dorado-0.3.2-linux-x64/lib/libaec.so.0

In [ ]:
import os
import shutil
from subprocess import Popen, PIPE, run, check_output
import gzip
import time
def exec(cmd,suppress_output=True):
    if suppress_output:
        out = run(cmd, stdout=PIPE, shell=True)
        return out.stdout.decode('utf-8'), out.stderr
    else:
        out = run(cmd, stdout=PIPE, shell=True)
        print (">>", cmd)
        print("Output:")
        print(out.stdout.decode('utf-8'))
        print("Exception:")
        print(out.stderr)
        return out.stdout.decode('utf-8'), out.stderr

In [ ]:
###Configure here###
DORADO_EXEC = "/content/dorado-0.3.2-linux-x64/bin/dorado"
#DORADO_EXEC = "/content/dorado-0.3.2-linux-x64/bin/dorado"
#Abs path of dorado
DORADO_EXEC = os.path.abspath(DORADO_EXEC)
ROOT = os.path.abspath("./")
#See here for available models  https://github.com/nanoporetech/dorado
#MODEL = "dna_r10.4.1_e8.2_400bps_sup@v4.2.0"
MODEL = "dna_r10.4.1_e8.2_400bps_sup@v4.2.0"
#MODEL = "dna_r9.4.1_e8_sup@v3.6"
RAW = "/content/NASData/NanoporeRawData/2023-000019/fast5/"
OUTPUT = "/content/NASData/Data/2023-000019/1_dorado/"
TMP = "./temp"
###Configure here###

In [ ]:
#Download the model
cmd = f"{DORADO_EXEC} download --model {MODEL} 2>&1 --directory {ROOT}"
print(cmd)
exec(cmd, suppress_output=False)

/content/dorado-0.3.2-linux-x64/bin/dorado download --model dna_r10.4.1_e8.2_400bps_sup@v4.2.0 2>&1 --directory /content
>> /content/dorado-0.3.2-linux-x64/bin/dorado download --model dna_r10.4.1_e8.2_400bps_sup@v4.2.0 2>&1 --directory /content
Output:

Exception:
None


('', None)

In [ ]:
#Count total raw files
total_raw = 0
for f in os.scandir(RAW):
    if f.name.endswith(".fast5") or f.name.endswith(".pod5"):
        total_raw += 1

parsed = 0
start_time = time.time()
for f in os.scandir(RAW):
    if f.name.endswith(".fast5") or f.name.endswith(".pod5"):
        #Check if output file exists
        os.makedirs(OUTPUT, exist_ok=True)
        output_fastq = os.path.join(OUTPUT, f.name+".fastq.gz")
        if os.path.exists(output_fastq):
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}] Skipping", f.name)
            total_raw -= 1
            continue
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}] Processing", f.name)
        #Copy fast5 to temp folder
        RAW_TMP_FOLDER = f"{TMP}/{f.name}"
        os.makedirs(f"{RAW_TMP_FOLDER}", exist_ok=True)
        shutil.copy(f.path, f"{RAW_TMP_FOLDER}")
        #Run dorado
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}] Basecalling")
        cmd = f"{DORADO_EXEC} basecaller {MODEL} {RAW_TMP_FOLDER} --emit-fastq > {RAW_TMP_FOLDER}/{f.name}.fastq"
        #print(cmd)
        exec(cmd)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}] Compressing")
        #compress fastq to fastq.gz
        with open(f"{RAW_TMP_FOLDER}/{f.name}.fastq", 'rb') as f_in:
            with open(f"{RAW_TMP_FOLDER}/{f.name}.fastq.gz", 'wb') as f_out:
                compressed_data = gzip.compress(f_in.read())
                f_out.write(compressed_data)
        shutil.move(f"{RAW_TMP_FOLDER}/{f.name}.fastq.gz", output_fastq)
        #wait for 5 seconds before deleting temp folder
        time.sleep(5)
        #Delete temp folder
        shutil.rmtree(RAW_TMP_FOLDER)
        #Print progress and estimated time
        parsed += 1
        elapsed_time = time.time() - start_time
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}] Processed {parsed}/{total_raw} ({parsed/total_raw*100:.2f}%). Est. time remaining: {elapsed_time/parsed*(total_raw-parsed):.2f} seconds")


[2024-03-11 07:31:37] Skipping FAU99894_309722fd_192e8acc_102.fast5
[2024-03-11 07:31:37] Skipping FAU99894_309722fd_192e8acc_103.fast5
[2024-03-11 07:31:38] Skipping FAU99894_309722fd_192e8acc_104.fast5
[2024-03-11 07:31:38] Skipping FAU99894_309722fd_192e8acc_105.fast5
[2024-03-11 07:31:38] Skipping FAU99894_309722fd_192e8acc_106.fast5
[2024-03-11 07:31:39] Skipping FAU99894_309722fd_192e8acc_107.fast5
[2024-03-11 07:31:39] Skipping FAU99894_309722fd_192e8acc_108.fast5
[2024-03-11 07:31:39] Skipping FAU99894_309722fd_192e8acc_109.fast5
[2024-03-11 07:31:40] Skipping FAU99894_309722fd_192e8acc_11.fast5
[2024-03-11 07:31:40] Skipping FAU99894_309722fd_192e8acc_110.fast5
[2024-03-11 07:31:41] Skipping FAU99894_309722fd_192e8acc_111.fast5
[2024-03-11 07:31:42] Skipping FAU99894_309722fd_192e8acc_112.fast5
[2024-03-11 07:31:42] Skipping FAU99894_309722fd_192e8acc_113.fast5
[2024-03-11 07:31:42] Skipping FAU99894_309722fd_192e8acc_114.fast5
[2024-03-11 07:31:43] Skipping FAU99894_309722fd_